# This file is used to produce summary statistics 

Based on data cleaned in Stata. 

In [1]:
datafolder = "C:\\Users\\xinxi\\Purdue\\EffectiveSocialLearningProject\\DataAnalysis\\ImportedData"
outputfolder = "C:\\Users\\xinxi\\Purdue\\EffectiveSocialLearningProject\\DataAnalysis\\Output_Figures" 

In [2]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import copy
import os
import random
import math as math
import matplotlib.pylab as plt #Typically matplotlib.pylab is imported as plt (shortened name)
import pdb
from scipy.optimize import minimize
import shutil
import csv
from MyLatexFunctions_SocialLearning import * 
from itertools import permutations 
from itertools import combinations
import csv


import networkx as nx

ModuleNotFoundError: No module named 'MyLatexFunctions_SocialLearning'

In [ ]:
data['earning'].mean()

In [ ]:
# Session and Treatment Summary
data = pd.read_csv(datafolder+"\\All_sessions_from_Stata.csv")
data_selected = data[['session', 'network_type', 'waiting_cost', 'date', 'earning', 'male', 'STEM','US_born']]
data_selected['network_type'] = data_selected['network_type'].replace(['a','b'], ['NC', 'C'])
data_selected['earning'] = data_selected['earning'].round(decimals=0)
df = data_selected.groupby(by=["session",'date', 'network_type', 'waiting_cost' ]).mean().reset_index()
df.columns = pd.MultiIndex.from_tuples([
     ("", "session"),
     ("", "date"),
    ("Treatment", "network_type"),
     ("Treatment", "waiting_cost"),
     ("Treatment", "earning"),
    ("Demographics", "male"),
    ("Demographics", "STEM"),
     ("Demographics", "US_born"),
])

# pd.options.display.float_format = '{:,.2f}'.format


df.rename(columns = {'earning': 'Earnings', 'session':'Session', 'date':'Date', 'network_type':'Network Type',
                    'waiting_cost': 'Waiting Cost', 'male':"% Male", 'STEM':'% STEM', 'US_born':"% U.S. Borned"}, inplace = True)


print(df.to_latex(column_format="cccccccc",index=False))
out_text = df.to_latex(column_format="cccccccc",index=False)
file_name = '01-SummaryStatistics-SummarybySession'
tex_file_name=file_name+".tex"
f = open(outputfolder+"\\"+tex_file_name, 'w')
f.write(out_text)
f.close()


    

In [ ]:
df = data_selected.groupby(by=['network_type', 'waiting_cost' ]).mean().reset_index()
df = df.drop(['session'], axis=1)
s = data_selected.groupby(by=['network_type', 'waiting_cost' ])['earning'].count().reset_index()
df = df.merge(s, on=['network_type', 'waiting_cost'],suffixes=('', '_r'))
df['Sessions'] = df['earning_r']/10
df = df.drop(['earning_r'], axis=1)
df['earning'] = df['earning'].round(decimals=2)
df['Sessions'] = df['Sessions'].astype(int)
df = df[['network_type', 'waiting_cost', 'Sessions',  'earning', 'male', 'STEM','US_born']]
df.columns = pd.MultiIndex.from_tuples([
    ("Treatment", "network_type"),
     ("Treatment", "waiting_cost"),
    ('Administration', 'Sessions'),
     ("Administration", "earning"),
    ("Demographics", "male"),
    ("Demographics", "STEM"),
     ("Demographics", "US_born"),
])

# pd.options.display.float_format = '{:,.2f}'.format


df.rename(columns = {'earning': 'Earnings', 'network_type':'Network Type',
                    'waiting_cost': 'Waiting Cost', 'male':"% Male", 'STEM':'% STEM', 'US_born':"% U.S. Borned"}, inplace = True)


print(df.to_latex(column_format="cccccccc",multicolumn_format='c'))
out_text = df.to_latex(column_format="cccccccc", multicolumn_format='c')
file_name = '01-SummaryStatistics-SummarybyTreatment'
tex_file_name=file_name+".tex"
f = open(outputfolder+"\\"+tex_file_name, 'w')
f.write(out_text)
f.close()


In [87]:
df = data_selected.groupby(by=['network_type', 'waiting_cost' ]).mean().reset_index()
df = df.drop(['session'], axis=1)

df

,network_type,waiting_cost,earning,male,STEM,US_born
0,C,H,23.05,65.0,85.0,40.0
1,C,L,24.84,50.0,56.0,70.0
2,NC,H,24.55,55.0,55.0,65.0
3,NC,L,24.92,50.0,60.0,67.5


In [68]:
treatment_data = data_selected.groupby(by=['network_type', 'waiting_cost' ])
var = 'earning'
r1 = treatment_data[var].mean()
r2 = treatment_data[var].std()
df_c = pd.concat([r1,r2.apply(lambda x: "("+"{:.2f}".format(x)+")")])
output = df_c.sort_index().rename(var)

for var in [ 'male', 'STEM','US_born']:
    
    r1 = treatment_data[var].mean()
    r2 = treatment_data[var].std()
    df_c = pd.concat([r1,r2.apply(lambda x: "("+"{:.2f}".format(x)+")")])
    df_cc = df_c.sort_index().rename(var)

    output = pd.concat([output, df_cc], axis=1)

In [67]:
output.join(r1, how="outer", rsuffix='right')

earning     male     STEM  US_born  US_bornright
network_type waiting_cost                                                 
C            H              23.05     65.0     85.0     40.0          40.0
             H             (2.54)  (48.94)  (36.63)  (50.26)          40.0
             L              24.84     50.0     56.0     70.0          70.0
             L             (2.11)  (50.51)  (50.14)  (46.29)          70.0
NC           H              24.55     55.0     55.0     65.0          65.0
             H             (1.73)  (51.04)  (51.04)  (48.94)          65.0
             L             24.925     50.0     60.0     67.5          67.5
             L             (1.97)  (50.64)  (49.61)  (47.43)          67.5

In [60]:
treatment_data = data_selected.groupby(by=['network_type', 'waiting_cost' ])
r1 = treatment_data[var].count()
r1

network_type  waiting_cost
C             H               20
              L               50
NC            H               20
              L               40
Name: US_born, dtype: int64

In [45]:
output


network_type  waiting_cost
C             H                23.05
              H               (2.54)
              L                24.84
              L               (2.11)
NC            H                24.55
              H               (1.73)
              L               24.925
              L               (1.97)
Name: earning, dtype: object